In [24]:
# 导入所需的package
import seaborn as sns #用于画图
from bs4 import BeautifulSoup #用于爬取arxiv的数据
import re #用于正则表达式，匹配字符串的模式
import requests #用于网络连接，发送网络请求，使用域名获取对应信息
import json #读取数据，我们的数据为json格式的
import pandas as pd #数据处理，数据分析
import matplotlib.pyplot as plt #画图工具

In [25]:
def readArxivFile(path, columns=['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi',
       'report-no', 'categories', 'license', 'abstract', 'versions',
       'update_date', 'authors_parsed'], count=None):
    '''
    定义读取文件的函数
        path: 文件路径
        columns: 需要选择的列
        count: 读取行数
    '''
    
    data  = []
    with open(path, 'r') as f: 
        for idx, line in enumerate(f): 
            if idx == count:
                break
                
            d = json.loads(line)
            d = {col : d[col] for col in columns}
            data.append(d)

    data = pd.DataFrame(data)
    return data

data = readArxivFile('arxiv-metadata-oai-snapshot.json', 
                     ['id', 'title', 'categories', 'abstract'],
                    200000)

In [26]:
# 为了方便数据的处理，我们可以将标题和摘要拼接一起完成分类
data['text'] = data['title'] + data['abstract']

data['text'] = data['text'].apply(lambda x: x.replace('\n',' '))
data['text'] = data['text'].apply(lambda x: x.lower())
data = data.drop(['abstract', 'title'], axis=1)

# 由于原始论文有可能有多个类别，所以也需要处理：
# 多个类别，包含子分类
data['categories'] = data['categories'].apply(lambda x : x.split(' '))
print(data['categories'])

# 单个类别，不包含子分类
data['categories_big'] = data['categories'].apply(lambda x : [xx.split('.')[0] for xx in x])
print(data['categories_big'])
print(data['categories_big'].iloc[:])

# 然后将类别进行编码，这里类别是多个，所以需要多编码：
# 多标签二值化：sklearn.preprocessing.MultiLabelBinarizer(classes=None, sparse_output=False)
# classes_属性：若设置classes参数时，其值等于classes参数值，否则从训练集统计标签值
# ①classes默认值，classes_属性值从训练集中统计标签值

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
data_label = mlb.fit_transform(data['categories_big'].iloc[:])
print(data_label)

0                             [hep-ph]
1                     [math.CO, cs.CG]
2                     [physics.gen-ph]
3                            [math.CO]
4                   [math.CA, math.FA]
                      ...             
199995    [astro-ph.CO, gr-qc, hep-th]
199996       [hep-ph, hep-ex, nucl-th]
199997            [cond-mat.stat-mech]
199998              [math.DG, math.AP]
199999             [cond-mat.mtrl-sci]
Name: categories, Length: 200000, dtype: object
0                          [hep-ph]
1                        [math, cs]
2                         [physics]
3                            [math]
4                      [math, math]
                    ...            
199995    [astro-ph, gr-qc, hep-th]
199996    [hep-ph, hep-ex, nucl-th]
199997                   [cond-mat]
199998                 [math, math]
199999                   [cond-mat]
Name: categories_big, Length: 200000, dtype: object
0                          [hep-ph]
1                        [math, cs]
2  

In [40]:
# 思路1
# 思路1使用TFIDF提取特征，限制最多4000个单词：
# tf−idf=tf(t,d)∗idf(t) tf(t,d) 表示在文本 d 中词项 t 出现的词数
# idf(t)=ln(1+df(d,t)/1+nd)+1 nd表示训练集文本数，df(d,t)表示包含词项t的文档总数
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=4000)
data_tfidf = vectorizer.fit_transform(data['text'].iloc[:])
print(data_tfidf)

# 由于这里是多标签分类，可以使用sklearn的多标签分类进行封装：
# 划分训练集和验证集
# 在机器学习中，我们通常将原始数据按照比例分割为“测试集”和“训练集”，从 sklearn.model_selection 中调用train_test_split 函数
# 简单用法如下：
# X_train,X_test, y_train, y_test =sklearn.model_selection.train_test_split(train_data,train_target,test_size=0.4, random_state=0,stratify=y_train)
# train_data：所要划分的样本特征集
# train_target：所要划分的样本结果
# test_size：样本占比，如果是整数的话就是样本的数量
# random_state：是随机数的种子。
# 随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。比如你每次都填1，其他参数一样的情况下你得到的随机数组是一样的。但填0或不填，每次都会不一样。
# stratify是为了保持split前类的分布。比如有100个数据，80个属于A类，20个属于B类。如果train_test_split(... test_size=0.25, stratify = y_all), 那么split之后数据如下：
# training: 75个数据，其中60个属于A类，15个属于B类。
# testing: 25个数据，其中20个属于A类，5个属于B类。
# 用了stratify参数，training集和testing集的类的比例是 A：B= 4：1，等同于split前的比例（80：20）。通常在这种类分布不平衡的情况下会用到stratify。
# 将stratify=X就是按照X中的比例分配
# 将stratify=y就是按照y中的比例分配
# 整体总结起来各个参数的设置及其类型如下：
# 主要参数说明：
# *arrays：可以是列表、numpy数组、scipy稀疏矩阵或pandas的数据框
# test_size：可以为浮点、整数或None，默认为None
# ①若为浮点时，表示测试集占总样本的百分比
# ②若为整数时，表示测试样本样本数
# ③若为None时，test size自动设置成0.25
# train_size：可以为浮点、整数或None，默认为None
# ①若为浮点时，表示训练集占总样本的百分比
# ②若为整数时，表示训练样本的样本数
# ③若为None时，train_size自动被设置成0.75
# random_state：可以为整数、RandomState实例或None，默认为None
# ①若为None时，每次生成的数据都是随机，可能不一样
# ②若为整数时，每次生成的数据都相同
# stratify：可以为类似数组或None
# ①若为None时，划分出来的测试集或训练集中，其类标签的比例也是随机的
# ②若不为None时，划分出来的测试集或训练集中，其类标签的比例同输入的数组中类标签的比例相同，可以用于处理不均衡的数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_tfidf, data_label,
                                                 test_size = 0.2, random_state = 1)
print("x_train:")                                                   
print(x_train)
print("x_test:")                                                     
print(x_test)
print("y_train:")                                                      
print(y_train)
print("y_test:")                                                    
print(y_test)

# 构建多标签分类模型
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
clf = MultiOutputClassifier(MultinomialNB()).fit(x_train, y_train)
print(clf)

  (0, 1338)	0.07713952249358953
  (0, 3234)	0.08325685818530902
  (0, 2529)	0.05055063593564344
  (0, 412)	0.02966358916483929
  (0, 541)	0.03558328143265654
  (0, 3309)	0.07173629935764876
  (0, 3247)	0.08000722643838502
  (0, 1277)	0.0793668877829128
  (0, 3639)	0.022452149048524696
  (0, 3300)	0.07940495295549563
  (0, 2846)	0.0652037095848
  (0, 2924)	0.0738235826562173
  (0, 3669)	0.060759033487410095
  (0, 478)	0.08277588025033003
  (0, 1752)	0.0817065496375506
  (0, 978)	0.06695254113468696
  (0, 686)	0.08582784041145053
  (0, 2076)	0.04678819616395626
  (0, 1273)	0.04492065018995425
  (0, 2849)	0.14685932651909842
  (0, 1152)	0.13723311109927955
  (0, 3301)	0.05462705182811686
  (0, 1159)	0.0671653764095451
  (0, 580)	0.10664045218016854
  (0, 3631)	0.085727271395169
  :	:
  (199999, 2344)	0.26599002995782295
  (199999, 1512)	0.10837249341120753
  (199999, 526)	0.03456567400311564
  (199999, 3971)	0.07918574429119493
  (199999, 2548)	0.033334257437417185
  (199999, 404)	0.06168

In [37]:
# sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
# 主要参数:
# y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。
# y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。
# labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。
# target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。
# sample_weight：类似于shape = [n_samples]的数组，可选项，样本权重。
# digits：int，输出浮点值的位数．
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.95      0.85      0.89      7925
           1       0.85      0.79      0.82      7339
           2       0.77      0.72      0.74      2944
           3       0.00      0.00      0.00         4
           4       0.72      0.48      0.58      2123
           5       0.51      0.66      0.58       987
           6       0.86      0.38      0.52       544
           7       0.71      0.69      0.70      3649
           8       0.76      0.61      0.68      3388
           9       0.85      0.88      0.87     10745
          10       0.46      0.13      0.20      1757
          11       0.79      0.04      0.07       729
          12       0.45      0.35      0.39       507
          13       0.54      0.36      0.43      1083
          14       0.69      0.14      0.24      3441
          15       0.84      0.20      0.33       655
          16       0.93      0.16      0.27       268
          17       0.87    

C:\Users\admin\AppData\Local\Programs\Python\Python38\Lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\AppData\Local\Programs\Python\Python38\Lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
# 思路2
# 思路2使用深度学习模型，单词进行词嵌入然后训练。将数据集处理进行编码，并进行截断：
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['text'].iloc[:100000], data_label[:100000],
                                                 test_size = 0.95, random_state = 1)
print("x_train:")                                                   
print(x_train)
print("x_test:")                                                     
print(x_test)
print("y_train:")                                                      
print(y_train)
print("y_test:")                                                    
print(y_test)

x_train:
81762    sign reversal of ac josephson current in a fer...
44595    mixed-symmetry massless fields in minkowski sp...
70766    spin relaxation of localized electrons in n-ty...
72830    numerical evidence for unstable magnons at hig...
50420    classical and quantized affine models of struc...
                               ...                        
50057    multicritical points for the spin glass models...
98047    spin-orbit-mediated spin relaxation in graphen...
5192     measurement of cp-violating asymmetries in b0-...
77708    the equilibrium intrinsic crystal-liquid inter...
98539    soft processes at the lhc, ii: soft-hard facto...
Name: text, Length: 5000, dtype: object
x_test:
43660    personal recommendation via modified collabora...
87278    noise driven translocation of short polymers i...
14317    uncertainty principle with quantum fisher info...
81932    sieving for mass equidistribution  we approach...
95321    critical formation of trapped surfaces in the ...

In [23]:
# parameter
max_features= 500
max_len= 150
embed_size=100
batch_size = 128
epochs = 5
# Tokenizer是一个用于向量化文本，或将文本转换为序列（即单词在字典中的下标构成的列表，从1算起）的类。
# Tokenizer实际上只是生成了一个字典，并且统计了词频等信息，并没有把文本转成需要的向量表示。
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

tokens = Tokenizer(num_words = max_features)
tokens.fit_on_texts(list(data['text'].iloc[:100000]))

print(tokens)
y_train = data_label[:100000]
x_sub_train = tokens.texts_to_sequences(data['text'].iloc[:100000])
print(x_sub_train)
x_sub_train = sequence.pad_sequences(x_sub_train, maxlen=max_len)

print("sequence:")
print( x_sub_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ssequence:
[[  0   0   0 ... 118  10   2]
 [  0   0   0 ...  14   4   4]
 [  0   0   0 ... 112  56  23]
 ...
 [  0   0   0 ...  72  12 374]
 [ 17 294  10 ...   2   3  57]
 [  0   0   0 ...  50 391   3]]


In [9]:
# 定义模型并完成训练：
# LSTM model
# Keras Layers:
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D# Keras Callback Functions:
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.models import Model
from keras.optimizers import Adam

sequence_input = Input(shape=(max_len, ))
x = Embedding(max_features, embed_size, trainable=True)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
preds = Dense(19, activation="sigmoid")(x)

model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
model.fit(x_sub_train, y_train, 
          batch_size=batch_size, 
          validation_split=0.2,
          epochs=epochs)

Epoch 1/5
625/625 [==============================] - 2013s 3s/step - loss: 0.2156 - accuracy: 0.4259 - val_loss: 0.1170 - val_accuracy: 0.6626
Epoch 2/5
625/625 [==============================] - 2081s 3s/step - loss: 0.1155 - accuracy: 0.6651 - val_loss: 0.1052 - val_accuracy: 0.6913
Epoch 3/5
625/625 [==============================] - 2150s 3s/step - loss: 0.1062 - accuracy: 0.6948 - val_loss: 0.1011 - val_accuracy: 0.6951
Epoch 4/5
625/625 [==============================] - 2350s 4s/step - loss: 0.1001 - accuracy: 0.7073 - val_loss: 0.0972 - val_accuracy: 0.7133
Epoch 5/5
625/625 [==============================] - 2186s 3s/step - loss: 0.0963 - accuracy: 0.7149 - val_loss: 0.0951 - val_accuracy: 0.7143
